## Data Cleaning

Let's load the some data for TSLA. Unfortunately, this data is not quite as *clean* as our NVDA data, so we'll need to do some data wrangling. The file we're looking to load is `TSLA_2015_2024.csv`.

In [139]:
import pandas as pd

In [1]:
df = pd.read_csv("../data/DASH_A1.csv")
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df.set_index("Date", inplace=True)
df.sort_index(inplace=True)
df


NameError: name 'pd' is not defined

## Duplicates 
Do we need this part?


In [141]:
print("Cleaning dataset of duplicates...")
print(f"Found {df.duplicated().sum()} duplicates.")
df.drop_duplicates(inplace=True)
print(f"There are now {df.duplicated().sum()} duplicates.")
# print("Dropped:", df.duplicated().sum())
print(f"There are {df.isnull().sum().sum()} values missing.")

Cleaning dataset of duplicates...
Found 50 duplicates.
There are now 0 duplicates.
There are 103 values missing.


## Cleaning rule 1
Open Prices: Missing values in the Open column are filled with the Close of the day before as an approximation, ignoring overnight trading.

In [142]:
df.isnull().sum() # Not sure if we should only display the open column

df.Open = df.Open.fillna(df.Close.shift(1))
df

df.isnull().sum() #  Not sure if we should only display the open column



Close     23
High      24
Low       15
Open       0
Volume    26
dtype: int64

## Cleaning rule 2
Close Prices: Missing values in the Close column are forward-filled to avoid look-ahead bias.


In [143]:
df.Close = df.Close.fillna(df.Open.shift(1))
df

df.isnull().sum() #  Not sure if we should only display the open column

df['Close'] = df['Close'].ffill()

## Cleaning rule 3
High and Low Prices: Missing values in the High and Low columns are filled with the mean of the respective High or Low within that month.

In [144]:
df.Volume.fillna(df.Volume.mean()) # use the mean to fill in missing value, does not reflect the trend, can't use mean over 10 years for every year

df.Close.interpolate(method="linear")

df.Close.ffill()


Date
2020-12-09    189.509995
2020-12-10    186.000000
2020-12-11    175.000000
2020-12-14    160.000000
2020-12-15    158.889999
                 ...    
2025-06-09    217.490005
2025-06-10    214.970001
2025-06-11    217.800003
2025-06-12    216.600006
2025-06-13    218.119995
Name: Close, Length: 1134, dtype: float64

#### Tip: Method Chaining

**Method chaining** is a popular feature of pandas. It allows us to *chain* together several operations in a single line of code. For example, we can set the index, sort the data frame and drop any duplicates all at once. Notice we don't use `inplace` but rather re-assign to the original `df` variable.

```python
df = df.set_index("Date", format="%d/%m/%Y).sort_index().drop_duplicates()
```

## Not a Number (NaN)

### Exercise: Some Null Chain

Let's look at the missing or `NaN` values next. Previously, we saw that `info()` gave us some insight into how many missing values we had, but we can also use `isnull()`.

Can you chain `isnull()` with `sum()` to get a single value stating the total number of missing values in the data frame?

In [145]:
df.isnull().sum()

Close      0
High      24
Low       15
Open       0
Volume    26
dtype: int64

We can find out which rows have missing data using `isnull()`, `any()` along rows and some smart *masking*.

In [146]:
condition = df.isnull().any(axis=1)
df[condition]

,Close,High,Low,Open,Volume
Date,,,,,
2020-12-10,186.000000,187.695007,172.636002,179.710007,NaN
2020-12-17,154.210007,161.660004,NaN,159.100006,6369800.0
2021-03-01,168.889999,173.789993,164.199997,171.440002,NaN
2021-03-15,140.690002,144.039993,NaN,143.779999,2107900.0
2021-03-16,133.660004,141.990005,NaN,140.789993,2477500.0
2021-04-26,153.800003,NaN,150.440002,153.199997,1563900.0
2021-05-25,136.270004,NaN,135.679993,140.110001,2812600.0
2021-06-24,175.300003,NaN,175.070007,176.110001,2682000.0
2021-07-09,174.979996,181.649994,176.539993,180.660004,NaN


Now that we've identified our missing values, the big question is how to handle them. There are many approaches to this that will vary depending on the data and the further analysis you plan to carry out.

In [147]:
df.dropna(subset="Close") # only drop missing values in close column

df.Volume.fillna(df.Volume.mean()) # use the mean to fill in missing value, does not reflect the trend, can't use mean over 10 years for every year

df.Close.interpolate(method="linear")

df.Close.ffill()


Date
2020-12-09    189.509995
2020-12-10    186.000000
2020-12-11    175.000000
2020-12-14    160.000000
2020-12-15    158.889999
                 ...    
2025-06-09    217.490005
2025-06-10    214.970001
2025-06-11    217.800003
2025-06-12    216.600006
2025-06-13    218.119995
Name: Close, Length: 1134, dtype: float64

### Exercise: Cleaning up

Notice how above we didn't actually update the `df` variable, so our DataFrame is still full of missing values. Fix all missing values applying the following rules:
- Fill missing Close by linear interpolation
- Fill missing Volume with the value from the day before
- Fill missing Open with the median Open
- Fill missing High with the Close or Open, whichever is higher
- Fill missing Low with a value 3% lower than the High


Your DataFrame `df` should have no missing values when done. Use `info()` to confirm.

**NOTE:** When changing values in a data frame, it is recommended to avoid using `inplace`, and instead re-assign the variable.

In [148]:
#1
df.Close = df.Close.interpolate(method="linear") # no need to use inplace in this method
#2
df.Volume = df.Volume.ffill()
#3
df.Open = df.Open.fillna (df.Open.median())
#4
df.High = df.High.fillna(df[["Close", "Open"]].max(axis=1))
#5
df.Low = df.Low.fillna(df.High*0.97)
df.isnull().sum().sum()
df


,Close,High,Low,Open,Volume
Date,,,,,
2020-12-09,189.509995,195.500000,163.800003,182.000000,25373700.0
2020-12-10,186.000000,187.695007,172.636002,179.710007,25373700.0
2020-12-11,175.000000,182.000000,168.250000,176.520004,4760600.0
2020-12-14,160.000000,170.000000,151.199997,169.100006,7859600.0
2020-12-15,158.889999,161.419998,153.759995,157.100006,5017000.0
...,...,...,...,...,...
2025-06-09,217.490005,219.830002,216.955002,218.029999,2710300.0
2025-06-10,214.970001,219.210007,210.927002,216.589996,3916700.0
2025-06-11,217.800003,219.529999,212.240005,214.184998,3091500.0


#### Advanced: Data Types

You may have noticed that the **Volume** column in the 2021 data frame is a `float64` instead of the `int64` *dtype* we had in the 2020 data frame. Missing values (NaN) are represented as a special case of floating point number, so all the values in **Volume** were automatically *upcast* to floats.

Ideally our columns should be of the *dtype* that most accurately represents them. This will improve performance when working with large data frames. Now that we've resolved our missing numbers, we can *cast* our trading volumes as integers.

In [149]:
df.info()
df.Volume = df.Volume.astype("int64")
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1134 entries, 2020-12-09 to 2025-06-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1134 non-null   float64
 1   High    1134 non-null   float64
 2   Low     1134 non-null   float64
 3   Open    1134 non-null   float64
 4   Volume  1134 non-null   float64
dtypes: float64(5)
memory usage: 53.2 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1134 entries, 2020-12-09 to 2025-06-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1134 non-null   float64
 1   High    1134 non-null   float64
 2   Low     1134 non-null   float64
 3   Open    1134 non-null   float64
 4   Volume  1134 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.2 KB


## Saving Data

Now that we've cleaned our data, let's save it, by writing it to a new .CSV file. We can use pandas' `to_csv()`.

In [150]:
df.to_csv("TSLA_clean.csv")